In [68]:
using QuantumCumulants
using IterTools

# Prototyping TCG Pole algorithm

## Theoretical background

In general, the TCG process requires calculating the contraction coefficients up to the order of truncation $k$,
$$\mathcal{C}_{l,r}(\vec\mu, \vec\nu) = \sum_{d\in \text{diagrams}} \frac{\tilde{f}\left(\lang \vec\mu_1 \rang +\lang \vec\nu_1 \rang\right) \tilde{f}\left(\lang \vec\mu_2 \rang +\lang \vec\nu_2 \rang\right) \cdots \tilde{f}\left(\lang \vec\mu_{||d||} \rang +\lang \vec\nu_{||d||} \rang\right)}{\vec\mu_1!\vec\nu_1!\cdot\vec\mu_2!\vec\nu_2!\cdot(\cdots)\cdot\vec\mu_{||d||}\vec\nu_{||d||}}$$
where the sum is over diagrams $d = [(l_1, r_1), (l_2, r_2), \cdots, (l_n, r_n)]$ where $n \le k$ and $k$ is the order of truncation and $(l_i, r_i)$ are tuples representing the bubbles in the diagram. The vectors $\vec\mu_i=(\mu_i^1, \mu_i^2, \cdots,\mu_i^{l_i})$ and $\vec\nu=(\nu_i^1, \nu_i^2, \cdots, \nu_i^{r_i})$ are the ordered frequencies of the modes in each bubble of the diagram. 
Additionally, we define the vector factorial function,
$$\vec \mu! = (\mu^1 + \mu^2 +\cdots + \mu^l)(\mu^1+\mu^2 + \cdots + \mu^{l-1})\cdots(\mu_1 + \mu_2)\mu_1 \\ \vec \nu! = (\nu^1 + \nu^2 +\cdots + \nu^l)(\nu^1 + \nu^2 + \cdots + \nu^{l-1})\cdots(\nu_1 + \nu_2)\nu_1$$

The problem with the TCG is that we can have resonances - points where some factors in the vector factorial vanish. Let us denote the sets of vanishing terms by $s$ and $s'$,
$$
\begin{matrix}
(\mu_i^{1} + \cdots+\mu_i^{s_1}) & (\mu_i^{1} + \cdots+\mu_i^{s_2}) &   \ldots & (\mu_i^{1} + \cdots+\mu_i^{s_n}) \\
(\nu_i^{1} + \cdots+ \nu_i^{s'_1}) & (\nu_i^{1} + \cdots + \nu_i^{s'_2}) &   \ldots & (\nu_i^{1} + \cdots + \nu_i^{s'_n})
\end{matrix}
$$
and also a corrected vector factorial without the vanishing terms,
$$
\overline{\vec\mu_i} = \prod_{j \notin s}^{l_i} (\mu_i^1 + \mu_i^2 + \cdots + \mu_i^j) \\
\overline{\vec\nu_i} = \prod_{j \notin s'}^{l_i} (\nu_i^1 + \nu_i^2 + \cdots + \nu_i^j)
$$

Through the power of math and friendship, the contribution of each diagram becomes,
$$
   \mathcal{E}_d = (-1)^{l}\prod_{i=1}^{||d||} \left [ -\frac{f(\lang \vec{\mu}_i \rang + \lang \vec{\nu}_i \rang)}{\overline{\vec\mu}_i! \overline{\vec\nu}_i!} \sum^{\sum_i (n_i + u_i + l_i) = \sum_i (|\vec s_i| + |\vec s_i'| )}_{n_i \ge 0, u_i \ge 0, l_i \ge 0} \sum_{k_i = 0}^{\lfloor \frac{n_i}{2} \rfloor} \frac{c(n_i, k_i)}{(n_i)!} \tau^{2(n_i - k_i)} \left(\lang \vec\mu_i \rang + \lang \vec\nu_i \rang \right)^{n_i} \\
    \sum_{\sum m_{J_u} = u_i, \sum m'_{J_l} = l_i} \frac{\prod_{J_u \notin \vec s_i, J_l \notin \vec s_i'}\left( \frac{-J_u}{\mu_i^1 + \cdots + \mu_i^{J_u}} \right)^{m_{J_u}} \left( \frac{-J_l}{\nu_i^1 + \cdots + \nu_i^{J_l}} \right)^{m'_{J_u}} }{\prod_{s_i^{j_1} \in \vec s_i, {s'}_i^{j_2} \in \vec{s'}_i} \left(s_i^{j_i} {s'}_i^{j_2} \right)} \right ]
$$

## Code and definitions

### Taylor coefficient

The Taylor expension can be calculated recursively,
$$ c(n+1,k)=-c(n,k) + (n-2k+2)c(n,k-1) $$

$$c(0,0) = 1 \\
c(n,-1) = 0 \\
c(n,k) \equiv 0 \text{ for }  n<2k$$

In [69]:
function taylor_coeff(n, k)
    # base cases
    if (k == -1) || (n < 2*k)
        return 0
    elseif (n == 0) && (k ==0)
        return 1
    else
        return -taylor_coeff(n - 1, k) + (n - 2*k + 1)*taylor_coeff(n - 1, k - 1)
    end
end

taylor_coeff (generic function with 1 method)

In [70]:
# base cases verification
@show taylor_coeff(1, -1)
@show taylor_coeff(231, -1)
@show taylor_coeff(3, 5)
@show taylor_coeff(3, 3)
println("---------")

# general cases
for i in 1:6
    for j in 1:6
        val = taylor_coeff(i, j)
        if val != 0  
            println("taylor_coeff($i, $j) = $(val)")
            println()
        end
    end
end

taylor_coeff(1, -1) = 0
taylor_coeff(231, -1) = 0
taylor_coeff(3, 5) = 0
taylor_coeff(3, 3) = 0
---------
taylor_coeff(2, 1) = -1

taylor_coeff(3, 1) = 3

taylor_coeff(4, 1) = -6

taylor_coeff(4, 2) = 3

taylor_coeff(5, 1) = 10

taylor_coeff(5, 2) = -15

taylor_coeff(6, 1) = -15

taylor_coeff(6, 2) = 45

taylor_coeff(6, 3) = -15



### Vector factorial

In [71]:
function vec_factorial(u; include_poles=true)
    temp_prod = u[1]
    for i = 2:length(u)
        temp_sum = sum(u[1:i])
        if isequal(temp_sum, 0)
            if include_poles == true
                temp_prod = 0
            end
        else
            temp_prod *= temp_sum
        end
    end
    return temp_prod
end

vec_factorial (generic function with 1 method)

In [72]:
a = [1, 3, 5, 7]
@show vec_factorial(a) 
@show 1*(1+3)*(1+3+5)*(1+3+5+7)

b = [1, 3, 5, 7, -(1+ 3 + 5 + 7)]
@show vec_factorial(b)
@show vec_factorial(b, include_poles=false)

vec_factorial(a) = 576
1 * (1 + 3) * (1 + 3 + 5) * (1 + 3 + 5 + 7) = 576
vec_factorial(b) = 0
vec_factorial(b, include_poles = false) = 576


576

In [73]:
@cnumbers a b c d

v = [a, b, c, -(a+b+c), d]
@show vec_factorial(v, include_poles=false)
@show vec_factorial(v, include_poles=true)


vec_factorial(v, include_poles = false) = a*d*(a + b)*(a + b + c)
vec_factorial(v, include_poles = true) = 0


0

### find_poles(u)

Finds all the factors of the vector factorial that evaluate to 0.

In [74]:
function find_poles(u)
    poles_list = []
    for i = 1:length(u)
        temp_sum = sum(u[1:i])
        if isequal(temp_sum, 0)
            append!(poles_list, i)
        end
    end
    return poles_list
end

find_poles (generic function with 1 method)

In [75]:
@cnumbers a b c d

v = [a, b, c, -(a+b+c), d, -d]

@show find_poles(v)

find_poles(v) = Any[4, 6]


2-element Vector{Any}:
 4
 6

Additionally, calculating the total number of poles requires finding the total number of poles. We define another function that recieves a vector of frequencies for a diagram and returns the total number of poles.

In [76]:
# ω = [(μ_1, ν_1), ..., (μ_||d||, ν_||d||)]
function find_all_poles(ω)
    μ_poles = []
    ν_poles = []
    for (μ, ν) in ω
        push!(μ_poles, find_poles(μ))
        push!(ν_poles, find_poles(ν))
    end
    return (μ_poles, ν_poles)
end

find_all_poles (generic function with 1 method)

In [77]:
μ1 = [a, b, -a-b] # pole at 3
ν1 = [a, -a] # pole at 2

μ2 = [0, a, a, -2a] # pole at 1 and 4
ν2 = [-a, b, a-b] # poles at 3
ω = [(μ1, ν1), (μ2, ν2)]

s_list, stag_list = find_all_poles(ω)

total_num_poles = sum([length(su) + length(sl) for (su, sl) in (s_list, stag_list)])

@show s_list
@show stag_list
@show total_num_poles

s_list = Any[Any[3], Any[1, 4]]
stag_list = Any[Any[2], Any[3]]
total_num_poles = 5


5

## Integer solutions

As we saw, our sums usually involve solving integer equations of the form $n_1 + n_2 + \cdots + n_k = m$ where all variables and parameters are integers.
It is useful then to have a function that efficiently solves this problem.

In [78]:
function find_integer_solutions(num_vars::Int, target_sum::Int, combination::Vector{Int}=Vector{Int}(), sum_so_far::Int=0)
    res = []
    if sum_so_far == target_sum
        if length(combination) == num_vars
            push!(res, combination)
            return res
        end
    elseif sum_so_far > target_sum
        return res
    end
    for i in 0:target_sum
        if length(combination) < num_vars
            res = vcat(res, find_integer_solutions(num_vars, target_sum, [combination..., i], sum_so_far+i))
        end
    end
    return res
end


find_integer_solutions (generic function with 3 methods)

In [79]:
function reshape_sols(sols, target_sum, num_bubbles, num_indices = 3)
    num_vars = num_bubbles*num_indices
    num_sols = floor(Int, factorial(num_vars + target_sum - 1)/(factorial(num_vars)*factorial(target_sum - 1)))
    dim_sols = num_sols                # total number of solutions
    dim_indices = num_indices          # number of indices - u, n, l
    dim_bubbles = num_bubbles          # number of bubbles 
    
    vectors = Array{Array{Int64,1}, 2}(undef, dim_sols, dim_bubbles)

    for i in 1:dim_sols           
        for j in 1:dim_bubbles    
            end_idx = dim_indices*j < length(sols[i][:]) ? dim_indices*j : length(sols[i][:])
            vectors[i, j] = sols[i][(1 + dim_indices*(j - 1)):end_idx]
        end
    end
    return vectors
end

reshape_sols (generic function with 2 methods)

Let us verify and try the following equations,
$$ n_1 = 5 $$
$$ n_1 + n_2 = 4 $$

In [80]:
@show find_integer_solutions(1, 5)
@show find_integer_solutions(2, 4)

find_integer_solutions(1, 5) = Any[[5]]
find_integer_solutions(2, 4) = Any[[0, 4], [1, 3], [2, 2], [3, 1], [4, 0]]


5-element Vector{Any}:
 [0, 4]
 [1, 3]
 [2, 2]
 [3, 1]
 [4, 0]

In [81]:
# 4 bubbles, each bubble with 3 indices that sum up to 4.
num_indices = 3
num_bubbles = 4
num_vars = num_indices*num_bubbles
target_sum = 4

sols = find_integer_solutions(num_vars, target_sum);
num_sols = floor(Int, factorial(num_vars + target_sum - 1)/(factorial(num_vars)*factorial(target_sum - 1)))

# reshape_sols(sols, target_sum, num_bubbles, num_indices = 3)

vecs = reshape_sols(sols, target_sum, num_bubbles)


@show size(vecs) 
@show num_sols
@show vecs

size(vecs) = (455, 4)
num_sols = 455
vecs = [[0, 0, 0] [0, 0, 0] [0, 0, 0] [0, 0, 4]; [0, 0, 0] [0, 0, 0] [0, 0, 0] [0, 1, 3]; [0, 0, 0] [0, 0, 0] [0, 0, 0] [0, 2, 2]; [0, 0, 0] [0, 0, 0] [0, 0, 0] [0, 3, 1]; [0, 0, 0] [0, 0, 0] [0, 0, 0] [0, 4, 0]; [0, 0, 0] [0, 0, 0] [0, 0, 0] [1, 0, 3]; [0, 0, 0] [0, 0, 0] [0, 0, 0] [1, 1, 2]; [0, 0, 0] [0, 0, 0] [0, 0, 0] [1, 2, 1]; [0, 0, 0] [0, 0, 0] [0, 0, 0] [1, 3, 0]; [0, 0, 0] [0, 0, 0] [0, 0, 0] [2, 0, 2]; [0, 0, 0] [0, 0, 0] [0, 0, 0] [2, 1, 1]; [0, 0, 0] [0, 0, 0] [0, 0, 0] [2, 2, 0]; [0, 0, 0] [0, 0, 0] [0, 0, 0] [3, 0, 1]; [0, 0, 0] [0, 0, 0] [0, 0, 0] [3, 1, 0]; [0, 0, 0] [0, 0, 0] [0, 0, 0] [4, 0, 0]; [0, 0, 0] [0, 0, 0] [0, 0, 1] [0, 0, 3]; [0, 0, 0] [0, 0, 0] [0, 0, 1] [0, 1, 2]; [0, 0, 0] [0, 0, 0] [0, 0, 1] [0, 2, 1]; [0, 0, 0] [0, 0, 0] [0, 0, 1] [0, 3, 0]; [0, 0, 0] [0, 0, 0] [0, 0, 1] [1, 0, 2]; [0, 0, 0] [0, 0, 0] [0, 0, 1] [1, 1, 1]; [0, 0, 0] [0, 0, 0] [0, 0, 1] [1, 2, 0]; [0, 0, 0] [0, 0, 0] [0, 0, 1] [2, 0, 1]; [0, 0, 0] [0

220×3 Matrix{Vector{Int64}}:
 [0, 0, 0]  [0, 0, 0]  [0, 0, 4]
 [0, 0, 0]  [0, 0, 0]  [0, 1, 3]
 [0, 0, 0]  [0, 0, 0]  [0, 2, 2]
 [0, 0, 0]  [0, 0, 0]  [0, 3, 1]
 [0, 0, 0]  [0, 0, 0]  [0, 4, 0]
 [0, 0, 0]  [0, 0, 0]  [1, 0, 3]
 [0, 0, 0]  [0, 0, 0]  [1, 1, 2]
 [0, 0, 0]  [0, 0, 0]  [1, 2, 1]
 [0, 0, 0]  [0, 0, 0]  [1, 3, 0]
 [0, 0, 0]  [0, 0, 0]  [2, 0, 2]
 ⋮                     
 [0, 1, 0]  [0, 0, 0]  [0, 1, 2]
 [0, 1, 0]  [0, 0, 0]  [0, 2, 1]
 [0, 1, 0]  [0, 0, 0]  [0, 3, 0]
 [0, 1, 0]  [0, 0, 0]  [1, 0, 2]
 [0, 1, 0]  [0, 0, 0]  [1, 1, 1]
 [0, 1, 0]  [0, 0, 0]  [1, 2, 0]
 [0, 1, 0]  [0, 0, 0]  [2, 0, 1]
 [0, 1, 0]  [0, 0, 0]  [2, 1, 0]
 [0, 1, 0]  [0, 0, 0]  [3, 0, 0]

### Diagram factor

In [146]:
function diagram_correction(ω)
    # Gets: ω is a list of tuples, including the frequency values for each bubble
    # e.g for a diagram d = [(3, 2), (2, 1)] we would ω = [(μ1, ν1), (μ2, ν2)] where μi and νi are vectors containing mode values.
	# Returns: a symbolic expression for the diagram correction
    @cnumbers τ
    f(x) = exp(-1/2*τ^2*x^2)

    # helper function for the innermost sum
    function norm_fac(v, j)
        if j == 0
            return 0
        else 
            return (j/sum(v[1:j]))
        end
    end

    # find all singularities
    num_bubbles = length(ω)
    (s_list, stag_list) = find_all_poles(ω)
    total_num_poles = sum([length(su) + length(sl) for (su, sl) in (s_list, stag_list)])

    bubble_factors = []                                                 # array holding the terms of the outer sum
    l_tot = 0
    unl_list = find_integer_solutions(3*length(ω), total_num_poles)     # first sum (n+u+l)_i = total_num_poles
    for (i, (μ, ν)) in enumerate(ω)
        l = length(μ)
        r = length(ν)
        l_tot += l
        @show (l, r)
        
        s = s_list[i, :] 
        stag = stag_list[i, :]                    # singular indices for current bubble
        num_poles = length(s) + length(stag)      # total number of poles

        ju_list = filter(x -> !(x in s), 1:l)
        jl_list = filter(x -> !(x in stag), 1:r)  # non-singular indices
        
        # finite part of the bubble factor (not including the expansion terms)
        prefac = -f(sum(μ) + sum(ν))/(vec_factorial(μ, include_poles = false)*vec_factorial(ν, include_poles=false))
        
        # sum over (n_i + u_i + l_i) = total_num_poles
        sols = find_integer_solutions(3*num_bubbles, total_num_poles)
        unl_list = reshape_sols(sols, total_num_poles, length(ω))
        inner_terms = []
        for (idx, (n, u, d)) in enumerate(unl_list[:, i])
            # first inner sum
            analytic_terms = [taylor_coeff(n, k)/factorial(n)*τ^(2*(n - k))*(sum(μ) + sum(ν))^(n - 2*k)*(d + r)^n for k = 0:floor(Int, n/2)]

            # second inner sum
            # mu_list and ml_list hold vectors of mu values
            pole_terms = []
            mu_list = find_integer_solutions(length(ju_list), u)
            ml_list = find_integer_solutions(length(jl_list), d)

            # denominator normalization factor
            denominator = prod([su*sl for (su, sl) in product(s_list[i], stag_list[i])])
            
            # numerator factors
            numerator = []
            for (mu_vec, ml_vec) in product(mu_list, ml_list)   # for each solution vector
                for (mu, ml) in product(mu_vec, ml_vec)         # for each value in the solution vector   
                    for (ju, jl) in product(ju_list, jl_list)   # for each non-singular factor
                        idx_u = indexin(ju, ju_list)[1]         # pick up the corresponding index for m_{Ju}
                        idx_l = indexin(jl, jl_list)[1]
                        push!(numerator, (norm_fac(μ, ju))^mu_vec[idx_u]*(norm_fac(ν, jl))^ml_vec[idx_l])
                    end
                    push!(pole_terms, prod(numerator)/prod(denominator))
                end
            end
            push!(inner_terms, sum(analytic_terms)*sum(pole_terms))
        end
        push!(bubble_factors, prefac*sum(inner_terms))
    end
    return (-1)^l_tot*prod(bubble_factors)
end

diagram_correction (generic function with 1 method)

In [145]:
#@cnumbers a, b, c
# μ = [0, a, -a] # pole at 3
# ν = [a, -a, b, c] # pole at 2

s = [1, 3]
stag = [2, 5, 7]                          # singular indices

ju_list = filter(x -> !(x in s), 1:3)
jl_list = filter(x -> !(x in stag), 1:4)  # non-singular indices
@show ju_list
@show jl_list

mu_list = find_integer_solutions(length(ju_list), 3)
ml_list = find_integer_solutions(length(jl_list), 4)

@show mu_list
@show ml_list

function norm_fac(v, j)
    if j == 0
        return 0
    else 
        return (j/sum(v[1:j]))
    end
end

numerator = []
for (mu_vec, ml_vec) in product(mu_list, ml_list)
    for (mu, ml) in product(mu_vec, ml_vec)
        # idx_u = indexin(mu, mu_vec)[1]
        # idx_l = indexin(ml, ml_vec)[1]
         
        # @show (mu, ml)
        # @show idx_u, idx_l

        facs = []
        for (ju, jl) in product(ju_list, jl_list)
            idx_u = indexin(ju, ju_list)[1]
            idx_l = indexin(jl, jl_list)[1]
            @show ju, jl
            @show mu_vec[idx_u], idx_u
            @show ml_vec[idx_l], idx_l
            push!(facs, (norm_fac(μ, ju))^mu_vec[idx_u]*(norm_fac(ν, jl))^ml_vec[idx_l])
        end
        push!(numerator, prod(facs))
    end
end

ju_list = [2]
jl_list = [1, 3, 4]
mu_list = Any[[3]]
ml_list = Any[[0, 0, 4], [0, 1, 3], [0, 2, 2], [0, 3, 1], [0, 4, 0], [1, 0, 3], [1, 1, 2], [1, 2, 1], [1, 3, 0], [2, 0, 2], [2, 1, 1], [2, 2, 0], [3, 0, 1], [3, 1, 0], [4, 0, 0]]
(ju, jl) = (2, 1)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (0, 1)
(ju, jl) = (2, 3)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (0, 2)
(ju, jl) = (2, 4)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (4, 3)
(ju, jl) = (2, 1)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (0, 1)
(ju, jl) = (2, 3)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (0, 2)
(ju, jl) = (2, 4)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (4, 3)
(ju, jl) = (2, 1)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (0, 1)
(ju, jl) = (2, 3)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (0, 2)
(ju, jl) = (2, 4)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (4, 3)
(ju, jl) = (2, 1)
(mu_vec[idx_


(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (0, 3)
(ju, jl) = (2, 1)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (3, 1)
(ju, jl) = (2, 3)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (1, 2)
(ju, jl) = (2, 4)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (0, 3)
(ju, jl) = (2, 1)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (4, 1)
(ju, jl) = (2, 3)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (0, 2)
(ju, jl) = (2, 4)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (0, 3)
(ju, jl) = (2, 1)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (4, 1)
(ju, jl) = (2, 3)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (0, 2)
(ju, jl) = (2, 4)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (0, 3)
(ju, jl) = (2, 1)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (4, 1)
(ju, jl) = (2, 3)
(mu_vec[idx_u], idx_u) = (3, 1)
(ml_vec[idx_l], idx_l) = (0, 2)
(ju, jl) = (2, 4)
(mu_vec[idx_u],

In [148]:
μ1 = [a, b, -a-b] # pole at 3
ν1 = [a, -a] # pole at 2

μ2 = [0, a, a, -2a] # pole at 1 and 4
ν2 = [-a, b, a-b] # poles at 3
ω = [(μ1, ν1), (μ2, ν2)]

s_list, stag_list = find_all_poles(ω)

total_num_poles = sum([length(su) + length(sl) for (su, sl) in (s_list, stag_list)])

diagram_correction(ω)

(l, r) = (3, 2)
(l, r) = (4, 3)


OverflowError: OverflowError: 1152921504606846976 * 32 overflowed for type Int64